# Use Beautiful Soup to Download Vans Data

- Define function

In [ ]:
# URL of vans shoes page

#create function to download images from a given url

def download_images(url,brand):

    vans_shoe_data = []

    for url in urls :
    # Send a GET request to the URL
        response = requests.get(url)

        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all shoe items on the page
        shoe_items = soup.find_all('div', class_='product-block')
        
        for i, shoe in enumerate(shoe_items):
            url = shoe.find('a', class_='product-block-pdp-url')['href']
            model = shoe.find('span', class_='product-block-name-wrapper').text.strip()
            price = shoe.find('span', class_='product-block-price product-block-offer-price offer-price product-price-amount-js').text.strip()
            
            # Extract shoe sizes and their corresponding measurements
            sizes = []
            size_elements = shoe.find_all('span', class_='css-1kj7abn')
            for size_element in size_elements:
                size_text = size_element.text.strip()
                # Extract the size and measurement separately
                size, measurement = size_text.split(' (')
                measurement = measurement[:-1]  # Remove trailing parenthesis
                sizes.append({'size': size, 'measurement': measurement})
            # Send a GET request to the URL
            response = requests.get(url)

            # Parse the HTML content of the page
            soup = BeautifulSoup(response.text, 'html.parser')
            images = soup.find_all('span',class_='mol-media-gallery-image-wrapper')
            # images.append(soup.find_all('div', class_='class="product-content-form-attr-container'))
            for j,image in enumerate(images) :
                url = image.find('img')['srcset']
                # Download images and save them into the files section
                image_name = f'/lakehouse/default/Files/vans_shoe_data/vans_shoe_{i}_{j}.jpg'
                try:
                    urllib.request.urlretrieve(url, image_name)
                except Exception as e:
                    print(f"Error downloading image {url}: {e}")  
                vans_shoe_data.append({'brand': brand, 'model': model, 'price': price, 'sizes': sizes, 'image_name': image_name})

                transform_image(image_name,i,j,brand,model,price,sizes,vans_shoe_data)

                adjust_brightness(image_name,i,j,brand,model,price,sizes,vans_shoe_data)
    import os
    import json

    # Save JSON data to files section
    output_dir = '/lakehouse/default/Files/vans_shoe_data/'
    os.makedirs(output_dir, exist_ok=True)
    json_file_path = os.path.join(output_dir, 'vans_shoe_data.json')
    with open(json_file_path, 'w') as f:
        json.dump(vans_shoe_data, f)

    # Display the extracted vans shoe data
    # print(vans_shoe_data)

    # Create directories if they don't exist
    os.makedirs('/lakehouse/default/Files/vans_shoe_data', exist_ok=True)

    # Save JSON data to files section
    with open('/lakehouse/default/Files/vans_shoe_data/vans_shoe_data.json', 'w') as f:
            f.write(json.dumps(vans_shoe_data))

In [ ]:
brand = 'Vans'
urls = ('https://www.vans.co.uk/shop/en-gb/vans-gb/women-shoes-hi-top-shoes',
        'https://www.vans.co.uk/shop/en-gb/vans-gb/women-shoes-classics',
        'https://www.vans.co.uk/shop/en-gb/vans-gb/women-shoes-low-top-shoes',
        'https://www.vans.co.uk/shop/en-gb/vans-gb/women-shoes-platform-shoes',
        'https://www.vans.co.uk/shop/en-gb/vans-gb/men-shoes-classics',
        'https://www.vans.co.uk/shop/en-gb/vans-gb/men-shoes-skate-shoes',
        'https://www.vans.co.uk/shop/en-gb/vans-gb/men-shoes-hi-top-shoes')


download_images(urls,brand)

In [ ]:
df = spark.read.json("Files/vans_shoe_data/vans_shoe_data.json")
# df now is a Spark DataFrame containing JSON data from Files.
display(df)

df.write.format("delta").mode("overwrite").save("Tables/shoe_details")